# AMSC698K Homework 9
##### Elijah Kin & Noorain Noorani

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pennylane as qml
from pennylane import numpy as pnp

### Problem A
It is comparatively easy to set up the QAOA circuits for MaxCut problems. Without using "black-box" QAOA implementations (classes and methods) create quantum circuits to perform a max.cut of a weighted undirected graph with 6 vertices using QISkit, Cirq, or Pennylane.

1. Compare the results (frequency plots) for 2, 4, and 6 layers.

In [ ]:
n_wires = 4
edges = [(0, 1), (0, 3), (1, 2), (2, 3)]

gr = nx.Graph(edges)
nx.draw(gr, with_labels=True, node_color="green")
plt.show()

In [ ]:
# unitary operator U_m (mixer) with parameter beta
def U_m(beta):
    for wire in range(n_wires):
        qml.RX(2 * beta, wires=wire)

# unitary operator U_c (cost) with parameter gamma
def U_c(gamma):
    for edge in edges:
        wire1 = edge[0]
        wire2 = edge[1]
        qml.CNOT(wires=[wire1, wire2])
        qml.RZ(gamma, wires=wire2)
        qml.CNOT(wires=[wire1, wire2])

In [ ]:
# TODO

2. Perform the optimization over 150 iterations for all three circuits (2-, 4-, 6-layers QAOA), compare the change of parameters and find the optimal number of iterations where the cost function is close to its minimum and the parameters don't change significantly.

In [ ]:
# TODO

3. Perform the optimization of the 6-layer circuit for a maximum of 80 iterations on the IonQ simulator with "aria-1" noise-model or on the IonQ QPU Aria-1 (remember to set the optional argument: error_mitigation={"debias":False} ).

In [ ]:
# TODO

### Problem B
Find the Minimum Cover for a graph with 8 vertices and 14 random edges (with at least 1 edge per vertex).

In [ ]:
# TODO

### Problem C
Suppose a restaurant offers food delivery within a distance of 3 miles. Set up the schedule for covering these four locations at the shortest overall distance.

Coordinates relative the restaurant's position: location 1 (-2.5 mi, 1.1 mi); location 2 (2.6 mi, -1.4 mi); location 3 (-1.8 mi, 2.2 mi); location 4 (1.1 mi, 1.2 mi).

In [ ]:
# TODO

### Problem D
Use QAOA to solve the following knapsack problem: suppose a car can carry freight up to 450 lb and you want to transport the highest-valued objects of a set of 12 items; which items to pick?

Weight and value of each item: A (51lb, $360); B (85lb, $180); C (40lb, $95); D (26lb, $130); E (64lb, $149); F (29lb, $69); G (65lb, $230); H (42lb, $199); I (48lb, $164); and J (36lb, $135); K (22lb, $109); L (46lb, $89).

In [ ]:
# TODO